In [1]:
!uv pip install feluda 
!uv pip install feluda-vid-vec-rep-clip 
!uv pip install feluda-dimension-reduction

Resolved 19 packages in 1.41s                                        
Prepared 17 packages in 1.50s                                            
Installed 17 packages in 27ms                               
 + boto3==1.35.96
 + botocore==1.35.96
 + certifi==2024.12.14
 + charset-normalizer==3.4.1
 + dacite==1.8.1
 + feluda==0.9.1
 + idna==3.10
 + jmespath==1.0.1
 + markupsafe==3.0.2
 + pydub==0.25.1
 + pyyaml==6.0.2
 + requests==2.32.3
 + s3transfer==0.10.4
 + toml==0.10.2
 + urllib3==2.3.0
 + werkzeug==3.1.3
 + wget==3.2
Resolved 39 packages in 1.26s                                        
Prepared 30 packages in 2m 07s                                           
Installed 30 packages in 97ms                               
 + feluda-vid-vec-rep-clip==0.1.1
 + filelock==3.16.1
 + fsspec==2024.12.0
 + huggingface-hub==0.27.1
 + jinja2==3.1.5
 + mpmath==1.3.0
 + networkx==3.4.2
 + numpy==2.2.1
 + nvidia-cublas-cu12==12.4.5.8
 + nvidia-cuda-cupti-cu12==12.4.127
 + nvidia-cuda-nvrtc-cu12==12.

In [2]:
!uv pip show feluda
!uv pip show feluda-vid-vec-rep-clip
!uv pip show feluda-dimension-reduction

Name: feluda
Version: 0.9.1
Location: /home/priyash7/Desktop/feluda/examples/.venv/lib/python3.10/site-packages
Requires: boto3, dacite, pydub, pyyaml, requests, toml, werkzeug, wget
Required-by:
Name: feluda-vid-vec-rep-clip
Version: 0.1.1
Location: /home/priyash7/Desktop/feluda/examples/.venv/lib/python3.10/site-packages
Requires: pillow, torch, torchvision, transformers
Required-by:
Name: feluda-dimension-reduction
Version: 0.1.1
Location: /home/priyash7/Desktop/feluda/examples/.venv/lib/python3.10/site-packages
Requires: numpy, scikit-learn
Required-by:


Using Python 3.10.12 environment at /home/priyash7/Desktop/feluda/.venv
Name: feluda-vid-vec-rep-clip
Version: 0.1.1
Location: /home/priyash7/Desktop/feluda/.venv/lib/python3.10/site-packages
Requires: pillow, torch, torchvision, transformers
Required-by:
Using Python 3.10.12 environment at /home/priyash7/Desktop/feluda/.venv
Name: feluda-dimension-reduction
Version: 0.1.1
Location: /home/priyash7/Desktop/feluda/.venv/lib/python3.10/site-packages
Requires: numpy, scikit-learn
Required-by:


In [4]:
from huggingface_hub import hf_hub_download
import tarfile
import cv2
from feluda import Feluda
import os
from feluda.models.media_factory import VideoFactory

/home/priyash7/Desktop/feluda/examples/.venv/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [5]:
# Downloading and extracting 

dataset_name = "UCF101_subset/train"
hf_dataset_identifier = "sayakpaul/ucf101-subset"
filename = "UCF101_subset.tar.gz"
file_path = hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset")


with tarfile.open(file_path) as t:
     t.extractall(".")

In [6]:
file_path

'/home/priyash7/.cache/huggingface/hub/datasets--sayakpaul--ucf101-subset/snapshots/b9984b8d2a95e4a1879e1b071e9433858d0bc24a/UCF101_subset.tar.gz'

In [14]:
feluda = Feluda(r'03_plot_tsne_videos_config.yml')
feluda.setup()

t-SNE model successfully initialized
Installing packages for vid_vec_rep_clip


In [8]:
dimension_reduction_operator = feluda.operators.get()[feluda.config.operators.parameters[0].type]
vid_vec_clip_operator = feluda.operators.get()[feluda.config.operators.parameters[1].type]

In [9]:
def get_video_thumbnail(video_path, save_path):
    """Extract and save the first frame from the video as a thumbnail."""
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()  # Read the first frame
    if ret:
        thumbnail_path = os.path.join(save_path, os.path.basename(video_path).replace('.avi', '_thumbnail.jpg'))
        cv2.imwrite(thumbnail_path, frame)  # Save the thumbnail as a JPEG
        cap.release()
        return thumbnail_path  # Return the path to the saved thumbnail
    cap.release()
    return None

In [10]:
operator_parameter = []
sub_folder_name = os.listdir(f'{dataset_name}')

for class_dir in sub_folder_name:
    temp_list = os.listdir(os.path.join(dataset_name, class_dir))

    i = 0
    while i < len(temp_list) and i < 5:
        if temp_list[i] == 'UCF101':
            i += 1 
            continue
            
        video_full_path = os.path.join(dataset_name, class_dir, temp_list[i])

        # Extract and save the video thumbnail before processing
        thumbnail_save_dir = 'thumbnails'
        os.makedirs(thumbnail_save_dir, exist_ok=True) 
        thumbnail_path = get_video_thumbnail(video_full_path, thumbnail_save_dir)
        
        video_path = VideoFactory.make_from_file_on_disk(
            os.path.join(dataset_name, class_dir, temp_list[i])
        )
        embedding = vid_vec_clip_operator.run(video_path)
        average_vector = next(embedding)
        
        operator_parameter.append({
            "payload": os.path.join(dataset_name, class_dir, temp_list[i]),  # Use the current item as the payload
            "embedding": average_vector.get("vid_vec"),
        })
        i += 1

IndexError: list index out of range

In [11]:
# Reducing dimension using operator

data = dimension_reduction_operator.run(operator_parameter)

ValueError: Input should be a non-empty list.

In [12]:
import numpy as np

thumbnail_folder = 'thumbnails'

x = np.array([item['reduced_embedding'][0] for item in data])
y = np.array([item['reduced_embedding'][1] for item in data])

def jitter(arr, jitter_amount=0.02):
    return arr + np.random.uniform(-jitter_amount, jitter_amount, arr.shape)

x_jittered = jitter(x, jitter_amount=0.3) 
y_jittered = jitter(y, jitter_amount=0.3)

plt.figure(figsize=(20, 16))
plt.scatter(x_jittered, y_jittered)

def load_thumbnail(payload):
    """Load the thumbnail from the pre-saved thumbnail folder."""
    video_filename = os.path.basename(payload) 
    thumbnail_filename = video_filename.replace('.avi', '_thumbnail.jpg')
    thumbnail_path = os.path.join(thumbnail_folder, thumbnail_filename)
    
    if os.path.exists(thumbnail_path):
        return cv2.imread(thumbnail_path)
    else:
        print(f"Thumbnail not found for {video_filename}")
    return None

for i, item in enumerate(data):
    video_thumbnail = load_thumbnail(item['payload'])
    
    if video_thumbnail is not None:
        video_thumbnail = cv2.resize(video_thumbnail, (100, 100))  # Smaller thumbnails
        video_thumbnail = cv2.cvtColor(video_thumbnail, cv2.COLOR_BGR2RGB)
        
        # Keep zoom as is, no need for offsets anymore
        imagebox = OffsetImage(video_thumbnail, zoom=0.5)
        ab = AnnotationBbox(imagebox, (x_jittered[i], y_jittered[i]), frameon=False)  # Use jittered values

        plt.gca().add_artist(ab)

# Set labels and title
plt.title('t-SNE Reduced Embeddings with Video Thumbnails')

plt.show()

NameError: name 'data' is not defined

In [13]:
# Cleaning cell

import shutil

shutil.rmtree('thumbnails')
shutil.rmtree('UCF101_subset')
os.remove(file_path)